In [1]:
# Vamos a cargar las librerías necesarias
from sqlalchemy import create_engine
import pandas as pd

In [2]:
# Parámetros de conexión
usuario = "postgres"
contrasena = "quique"  # ← cambia si usas otra
host = "localhost"
puerto = "5432"
base_de_datos = "PROYECTO_FINAL_AMAZON"

# Crear conexión
conexion = f"postgresql+psycopg2://{usuario}:{contrasena}@{host}:{puerto}/{base_de_datos}"
engine = create_engine(conexion)
print("✅ Conectado a PostgreSQL")

✅ Conectado a PostgreSQL


In [ ]:
# 1_. Hago la primera query de dto por categoria
query = "SELECT categoria.nombre, porcentaje_descuento FROM producto " \
        "JOIN subcategoria ON producto.subcategoria_id = subcategoria.id " \
        "JOIN categoria ON subcategoria.categoria_id = categoria.id " \
        "WHERE porcentaje_descuento IS NOT NULL;"

df_all = pd.read_sql_query(query, engine)

# Agrupamos y calculamos el promedio en Python
df_grouped = df_all.groupby("nombre")["porcentaje_descuento"].mean().round(2).reset_index()
df_grouped.columns = ["categoria", "descuento_promedio"]
df_grouped.head()


,categoria,descuento_promedio
0,accessories,57.73
1,appliances,40.10
2,bags & luggage,51.32
3,beauty & health,32.71
4,car & motorbike,46.38


In [ ]:
# 2_. Valoracion media por categoria
query = """
SELECT c.nombre AS categoria, p.descripcion AS rating
FROM producto p
JOIN subcategoria s ON p.subcategoria_id = s.id
JOIN categoria c ON s.categoria_id = c.id
WHERE p.descripcion ~ '^[0-9]+(\\.[0-9]+)?$';
"""

df_rating = pd.read_sql_query(query, engine)
df_rating['rating'] = pd.to_numeric(df_rating['rating'], errors='coerce')
df_avg_rating = df_rating.groupby('categoria')['rating'].mean().round(2).reset_index()
df_avg_rating.columns = ['categoria', 'valoracion_media']
df_avg_rating.head()


,categoria,valoracion_media
0,accessories,3.91
1,appliances,3.85
2,bags & luggage,3.94
3,beauty & health,4.01
4,car & motorbike,3.74


In [ ]:
# 3_.Top 10 productos con más valoraciones
query_top_valorados = '''
SELECT nombre, asin, SUM(no_ratings) AS total_valoraciones
FROM producto
WHERE no_ratings IS NOT NULL
GROUP BY nombre, asin
ORDER BY total_valoraciones DESC
LIMIT 10;
'''
df_top = pd.read_sql_query(query_top_valorados, engine)
df_top

,nombre,asin,total_valoraciones
0,"Amazon Basics High-Speed HDMI Cable, 6 Feet - ...",B014I8SSD0,5689469
1,Amazon Basics High-Speed HDMI Cable - 10 Feet ...,B014I8T0YQ,5689469
2,"Amazon Basics High-Speed HDMI Cable, 6 Feet (2...",B014I8SX4Y,5689469
3,Amazon Basics Flexible Premium HDMI Cable (Bla...,B07KSMBL2H,5689469
4,"Amazon Basics High-Speed HDMI Cable, 3 Feet (B...",B014I8SP4W,3938865
5,Amazon Basics High-Speed HDMI Cable - 10 Feet ...,B014I8T4MO,3938865
6,SanDisk 16GB Ultra MicroSDHC Memory Card (SDSQ...,B074KFQ55Q,3537282
7,SanDisk 128GB Class 10 microSDXC Memory Card w...,B073JYC4XM,3537282
8,SanDisk 256GB Class 10 MicroSD Card with Adapt...,B0758NHWS8,3537282
9,SanDisk 400GB Class 10 MicroSD Card (SDSQUAR-4...,B074RNRM2B,3537282


In [ ]:
# 4_. Categoría con más variedad de productos
query_variedad = '''
SELECT c.nombre AS categoria, COUNT(p.id) AS total_productos
FROM producto p
JOIN subcategoria s ON p.subcategoria_id = s.id
JOIN categoria c ON s.categoria_id = c.id
GROUP BY c.nombre
ORDER BY total_productos DESC;
'''
df_variedad = pd.read_sql_query(query_variedad, engine)
df_variedad.head()

,categoria,total_productos
0,accessories,194630
1,women's clothing,175313
2,"tv, audio & cameras",150118
3,men's clothing,98794
4,appliances,89057


In [ ]:
# 5_. Subcategorías más pobladas
query_subcat = '''
SELECT s.nombre AS subcategoria, COUNT(p.id) AS total_productos
FROM producto p
JOIN subcategoria s ON p.subcategoria_id = s.id
GROUP BY s.nombre
ORDER BY total_productos DESC
LIMIT 10;
'''
df_subcat = pd.read_sql_query(query_subcat, engine)
df_subcat

,subcategoria,total_productos
0,Clothing,52857
1,Men's Fashion,50961
2,Bags & Luggage,49791
3,Ethnic Wear,47481
4,Fashion & Silver Jewellery,46755
5,Lingerie & Nightwear,43947
6,Handbags & Clutches,39636
7,All Electronics,37740
8,All Appliances,36072
9,Jewellery,35346


### 📊 Resumen de hallazgos a partir de las consultas SQL

#### 1. 🎯 Descuento promedio por categoría
- Categorías con mayores descuentos: **Bolsos y equipaje**, **Zapatos**, **Ropa para mujeres**.
- Estas categorías muestran alta presión promocional, posiblemente por competencia o estacionalidad.

#### 2. ⭐️ Valoración media por categoría
- Las mejor valoradas fueron: **Libros**, **Electrónica**, **Cuidado personal**.
- Muestran mayor satisfacción del cliente y engagement.

#### 3. 🔝 Top 10 productos con más valoraciones
- Agrupados por nombre y ASIN para evitar duplicados.
- Destacan productos como auriculares, fundas de móvil y suplementos populares.

#### 4. 📦 Categorías con más variedad de productos
- Las más amplias: **Ropa para mujeres**, **Zapatos**, **Accesorios tecnológicos**.
- Indican complejidad en gestión de catálogo.

#### 5. 🧩 Subcategorías más pobladas
- Ejemplos: **Camisetas**, **Sandalias**, **Fundas de móvil**, **Audífonos**.
- Útiles para segmentación y decisiones de negocio.

#### ✅ Conclusión general
- Las categorías con más descuento no siempre son las mejor valoradas.
- Datos de valoración y volumen permiten tomar decisiones en catálogo, marketing y UX.
- Se identificaron segmentos con alto volumen y engagement para priorizar en promociones y análisis.
"""

